### import yelp databases, clean, organize and merge into final databases.

In [69]:
import pandas as pd

In [70]:
#import data csv
spa_data = pd.read_csv('finalspadata.csv')
rest_data = pd.read_csv('finaldata.csv')  
hotel_data = pd.read_csv('finalhoteldata.csv')

In [71]:
#convert price to numeric value
def cleandata(data):
    data = data.drop(columns={'Unnamed: 0', 'BusinessType'})
    data['price']=data['price'].apply(lambda x: 1 if (x == "€") else x)
    data['price']=data['price'].apply(lambda x: 2 if (x == "€€") else x)
    data['price']=data['price'].apply(lambda x: 3 if (x == "€€€") else x)
    data['price']=data['price'].apply(lambda x: 2 if (x == "££") else x)
    data['price']=data['price'].apply(lambda x: 1 if (x == "$") else x)
    data['price']=data['price'].apply(lambda x: 2 if (x == "$$") else x)
    data['price']=data['price'].apply(lambda x: 3 if (x == "$$$") else x)
    data['price']=data['price'].apply(lambda x: 4 if (x == "$$$$") else x)
    data['price'] = data['price'].astype('int')
    data['City'] = data['City'].apply(lambda x: x.strip())
    data['State'] = data['State'].apply(lambda x: x.strip())
    
    #drop 0 values from database
    data = data.loc[data['price'] != 0]
    
    return data


In [72]:
#call function to clean data
spa_data = cleandata(spa_data)
rest_data = cleandata(rest_data)
hotel_data = cleandata(hotel_data)

In [73]:
rest_data

,State,City,name,cat,price,rating,review,Total
1,al,butler,Don Rafa,Mexican,2,5.0,4,6
5,al,butler,McDonald's,Coffee & Tea,1,1.0,3,6
8,al,cherokee,Point Of View Restaurant And Oyster Bar,Seafood,2,4.0,2,8
9,al,cherokee,Rattlesnake Saloon,Bars,1,3.5,50,8
10,al,cherokee,The Southern Farm Table Restaurant and Bakery,American (Traditional),2,4.5,14,8
...,...,...,...,...,...,...,...,...
65751,id,boise,Jimmy John's,Sandwiches,1,3.0,21,931
65752,id,boise,Wingstop,Chicken Wings,1,3.0,60,931
65753,id,boise,Fish Poké Bar,Poke,2,4.0,33,931
65754,id,boise,Louie's Pizza and Italian Restaurant,Pizza,2,3.5,114,931


In [74]:
new_york = rest_data.loc[rest_data['City'] =='new york']

In [75]:
#make a seperate database of expensive and cheap for business type for seperate analysis
exp_spa_data = spa_data.loc[spa_data['price'] >= 3]
cheap_spa_data = spa_data.loc[spa_data['price'] < 3]
exp_rest_data = rest_data.loc[rest_data['price'] >= 3]
cheap_rest_data = rest_data.loc[rest_data['price'] < 3]
exp_hotel_data = hotel_data.loc[hotel_data['price'] >= 3]
cheap_hotel_data = hotel_data.loc[hotel_data['price'] < 3] 

In [76]:
#read in PCI and pop data, clean and sort
other_db = pd.read_csv("final_before_yelp.csv")
other_db = other_db.drop(columns={'Unnamed: 0'})
other_db = other_db.rename(columns={'DataValue':'PCI'})
other_db = other_db.sort_values('State')
other_db.reset_index(drop=True, inplace=True)
other_db['City'] = other_db['City'].apply(lambda x: x.lower())
other_db['State'] = other_db['State'].apply(lambda x: x.lower())
other_db['PCI'] = other_db['PCI'].apply(lambda x: x.replace(',', ''))
other_db['PCI'] = other_db['PCI'].astype('int')


In [77]:
new_york_merge = other_db.merge(new_york, on=['State', 'City'])

In [78]:
new_york_merge.drop(['State', 'City', 'name', 'Total'], axis=1, inplace=True)

In [79]:
new_york_merge.to_csv('new_york_merge.csv', index=False)

In [80]:
#function that creates a count of each price point and merges all databases

def buscount(data):
    
    data_means = data.groupby(['State', 'City']).mean()
    count = data.groupby(['State', 'City']).count()['name']
    data_count = pd.DataFrame(count)
    data_count.rename(columns={'name':'P total'}, inplace=True)
    data1 = data_count.merge(data_means, on=['State', 'City'], how='inner')
    data1['Price perc'] = data1['P total'] / data1['Total']    
    data1.reset_index(inplace=True)
    data1.drop(['Total', 'P total'], axis=1, inplace=True)
    
    final = other_db.merge(data1, how='inner', on=['State', 'City'])
    return final

In [81]:
#function that process for full data
def fullcount(data):
    
    data_means = data.groupby(['State', 'City']).mean()
    count = data.groupby(['State', 'City']).count()['name']
    data_count = pd.DataFrame(count)
    data_count.rename(columns={'name':'P total'}, inplace=True)
    data1 = data_count.merge(data_means, on=['State', 'City'], how='inner')
    data1.reset_index(inplace=True)
    final = other_db.merge(data1, how='inner', on=['State', 'City'])
    final['PP'] = final['P total'] / final['Pop']    

    final.drop(['Total', 'P total'], axis=1, inplace=True)
    
    return final

In [82]:
exp_spa_data = buscount(exp_spa_data)
cheap_spa_data = buscount(cheap_spa_data)
exp_rest_data = buscount(exp_rest_data)
cheap_rest_data = buscount(cheap_rest_data)
exp_hotel_data = buscount(exp_hotel_data)
cheap_hotel_data= buscount(cheap_hotel_data)
full_hotel_data = fullcount(hotel_data)
full_rest_data = fullcount(rest_data)

In [84]:
full_hotel = full_hotel_data.merge(full_rest_data, on=['State', 'City'])

In [85]:
full_hotel

,State,City,Pop_x,PCI_x,price_x,rating_x,review_x,PP_x,Pop_y,PCI_y,price_y,rating_y,review_y,PP_y
0,ak,bristol bay borough,877,116340,2.666667,4.666667,5.000000,0.003421,877,116340,1.500000,3.750000,6.500000,0.002281
1,ak,sitka city and borough,8647,66157,2.200000,3.650000,13.500000,0.001156,8647,66157,1.933333,3.966667,38.466667,0.001735
2,ak,matanuska-susitna borough,107610,45608,2.083333,4.083333,36.416667,0.000112,107610,45608,1.894737,3.947368,68.421053,0.000177
3,ak,kenai peninsula borough,58533,50466,2.272727,4.181818,7.818182,0.000188,58533,50466,2.111111,4.277778,64.166667,0.000308
4,ak,juneau city and borough,32113,62774,2.200000,3.266667,20.066667,0.000467,32113,62774,1.911765,3.911765,110.058824,0.001059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,wv,morgan,30955,30670,1.875000,3.437500,16.725000,0.001292,30955,30670,1.000000,2.909091,4.363636,0.000355
1193,wv,mason,938,28756,1.714286,3.321429,13.285714,0.014925,938,28756,1.083333,3.500000,5.833333,0.012793
1194,wv,logan,1511,32521,2.333333,3.333333,7.000000,0.001985,1511,32521,1.222222,4.333333,4.555556,0.005956
1195,wy,laramie,32473,49426,1.736842,3.421053,22.736842,0.000585,32473,49426,1.566667,3.583333,72.300000,0.000924


In [86]:
exp_spa_data.to_csv('exp_spa_data.csv', index=False)
cheap_spa_data.to_csv('cheap_spa_data.csv', index=False)
exp_rest_data.to_csv('exp_rest_data.csv', index=False)
cheap_rest_data.to_csv('cheap_rest_data.csv', index=False)
exp_hotel_data.to_csv('exp_hotel_data.csv', index=False) 
cheap_hotel_data.to_csv('cheap_hotel_data.csv', index=False)
full_hotel.to_csv('full_hotel_data.csv', index=False)

In [36]:
high_pop = final_db[(final_db['Pop'] > 60000)]
low_pop = final_db[(final_db['Pop'] < 60000)]


In [61]:
high_pop = high_pop.merge(data1_means, how='inner', on=['State', 'City'])
low_pop = low_pop.merge(data1_means, how='inner', on=['State', 'City'])

In [64]:
hp_f = final_db.merge(hp_final, how='inner', on=['State', 'City'])

In [252]:
final_final = final_db.merge(data1_means, how='inner', on=['State', 'City'])

In [62]:
high_pop.to_csv('high_pop.csv')
low_pop.to_csv('low_pop.csv')